In [1]:
import pandas as pd

In [2]:
import nosqlite

In [3]:
conn = nosqlite.Connection("../data/results.db")

In [4]:
uminho = conn["uminho"]

In [5]:
uminho.count()

85

In [6]:
template = uminho.find_one()

In [7]:
class Bag(list):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def pluck(self, key, default=None):
        return Bag(doc.get(key, default) for doc in self)

In [8]:
bag = Bag(uminho.find())

In [9]:
conn.close()

In [10]:
len(bag)

85

In [11]:
template["report"]["test_basic"]["test_model_id_presence"].keys()

dict_keys(['data', 'duration', 'message', 'metric', 'result', 'summary', 'title', 'type'])

In [12]:
tests = list()
titles = list()
summaries = list()
metrics = list()
for module in template["report"]:
    for test in template["report"][module]:
        if template["report"][module][test].get("type", "") == "object":
            print("Skipping", test)
            continue
        t = template["report"][module][test].get("title", "")
        if t == "":
            print(test, "has no title.")
            continue
        tests.append(test)
        titles.append(t)
        summaries.append(template["report"][module][test].get("summary", ""))
        metrics.append(bag.pluck("report", {}).pluck(module, {}).pluck(test, {}).pluck("metric", float("nan")))

Skipping test_metabolite_annotation_overview
Skipping test_metabolite_annotation_wrong_ids
Skipping test_reaction_annotation_overview
Skipping test_reaction_annotation_wrong_ids
Skipping test_biomass_consistency
Skipping test_biomass_default_production
Skipping test_biomass_precursors_default_production
Skipping test_biomass_precursors_open_production
Skipping test_fast_growth_default
Skipping test_gam_in_biomass
Skipping test_detect_energy_generating_cycles
test_detect_energy_generating_cycles[glu__L_c] has no title.


In [13]:
meta = pd.DataFrame({"test_case": tests, "title": titles, "summary": summaries})

In [14]:
metric = pd.DataFrame({case: m for case, m in zip(tests, metrics)})

In [15]:
metric["source"] = "U Minho"

In [16]:
metric["model_id"] = bag.pluck("model_id", "")

In [17]:
metric.head()

,test_biomass_presence,test_blocked_reactions,test_compartments_presence,test_enzyme_complex_presence,test_find_deadends,test_find_disconnected,test_find_orphans,test_find_pure_metabolic_reactions,test_find_stoichiometrically_balanced_cycles,test_find_transport_reactions,...,test_ngam_presence,test_reaction_annotation_presence,test_reaction_charge_balance,test_reaction_id_namespace_consistency,test_reaction_mass_balance,test_reactions_presence,test_stoichiometric_consistency,test_transport_reaction_presence,source,model_id
0,1.0,0.204931,1.0,1.0,0.071501,0.0,0.110775,0.695686,1.0,0.159476,...,1.0,1.0,0.017720,1.0,0.006934,1.0,0.830816,1.0,U Minho,
1,1.0,0.194253,1.0,1.0,0.047686,0.0,0.040673,0.889655,1.0,0.000000,...,1.0,1.0,0.008046,1.0,0.000000,1.0,0.000000,1.0,U Minho,July2009_Model
2,1.0,0.271772,1.0,1.0,0.086957,0.0,0.076292,0.957958,1.0,0.000000,...,1.0,1.0,0.957958,1.0,0.000000,1.0,0.662838,1.0,U Minho,iSS884
3,1.0,0.465162,1.0,1.0,0.189693,0.0,0.134868,0.999019,1.0,0.000000,...,1.0,1.0,0.999019,1.0,0.000000,1.0,0.142544,1.0,U Minho,HyunUkKim2010_VvuMBEL943_MetabolicModeling
4,1.0,0.069794,1.0,1.0,0.021179,0.0,0.014273,0.917864,1.0,0.070691,...,1.0,1.0,0.000000,1.0,0.001571,1.0,0.001381,1.0,U Minho,pti


In [18]:
from csv import QUOTE_NONNUMERIC

In [19]:
metric.to_csv("../data/uminho/metrics.csv", index=False, quoting=QUOTE_NONNUMERIC)